# Python for Finance - part 8

#### 금융분석과 관련된 기본적인 내용을 정리하였습니다. 파이썬을 활용한 금융분석 (이브 힐피시 지음)을 참고하였습니다.

## Chapter 15 가치 평가 프레임워크
### 15.1 자산 가격결정 기본 정리
#### 자산 가격결정 기본 정리 : 마틴게일 측도가 핵심, 할인된 위험 요인에서 실제 증가율을 제거한 확률 측도, 마틴게일 측도 하에서 모든 위험 요인은 위험 프리미엄을 포함한 실제 시장 이자율을 무시하고 무조건 고정된 무위험 단기 이자율로 증가 

#### 15.1.1 간단한 예제
#### 15.1.2 일반적인 결과
#### 조건이 아무리 복잡해도 모두 적용 가능하다는 장점이 있음
#### 마틴 게일 측도 하에서는 모든 위험 자산이 무위험 단기 이자율로 증가하기 때문에 '위험 중립 가치 평가 방법'이라고도 불림

### 15.2 위험 중립 할인
#### 15.2.1 날짜 모형과 조작
#### 날짜의 목록을 만들기 위한 방법은 2가지 : 구체적인 날짜의 리스트 / 1년을 기준으로 한 비율 숫자로 표현

In [2]:
import datetime as dt
dates = [dt.datetime(2015, 1, 1), dt.datetime(2015, 7,1), dt.datetime(2016, 1, 1)]
(dates[1] - dates[0]).days / 365.

0.4958904109589041

In [3]:
(dates[2] - dates[1]).days / 365.

0.5041095890410959

In [4]:
fractions = [.0, .5, 1.]

In [5]:
import numpy as np
def get_year_deltas(date_list, day_count=365.):
    """ 날짜를 1년 비율로 표현한 부동소수점 벡터를 반환"""
    
    start = date_list[0]
    delta_list = [(date - start).days / day_count for date in date_list]
    return np.array(delta_list)

In [6]:
dates = [dt.datetime(2015, 1, 1), dt.datetime(2015, 7,1), dt.datetime(2016, 1, 1)]
get_year_deltas(dates)

array([0.        , 0.49589041, 1.        ])

#### 15.2.2 고정 단기 이자율

In [7]:
class constant_short_rate(object):
    """ 고정 단기 이자율 할인 클래스 """
    
    def __init__(self, name, short_rate):
        self.name = name
        self.short_rate = short_rate
        if short_rate < 0 :
            raise ValueError('Short rate negative')
            
    def get_discount_factors(self, date_list, dtobjects=True):
        """ datetime 객체 또는 1년 기준 비율의 리스트/배열이 주어지면 할인율을 반환 """
        if dtobjects:
            dlist = get_year_deltas(date_list)
        else:
            dlist = np.array(date_list)
        dflist = np.exp(self.short_rate * np.sort(-dlist))
        return np.array((date_list, dflist)).T

In [8]:
dates = [dt.datetime(2015, 1, 1), dt.datetime(2015, 7,1), dt.datetime(2016, 1, 1)]
csr = constant_short_rate('csr', .05)
csr.get_discount_factors(dates)

array([[datetime.datetime(2015, 1, 1, 0, 0), 0.951229424500714],
       [datetime.datetime(2015, 7, 1, 0, 0), 0.9755103387657228],
       [datetime.datetime(2016, 1, 1, 0, 0), 1.0]], dtype=object)

In [11]:
deltas = get_year_deltas(dates)
csr.get_discount_factors(deltas, dtobjects = False)

array([[0.        , 0.95122942],
       [0.49589041, 0.97551034],
       [1.        , 1.        ]])

### 15.3 시장 환경

In [14]:
class market_environment(object):
    """ 가치 평가와 관련된 시장 환경 모형을 위한 클래스 """
    
    def __init__(self, name, pricing_date):
        """
        :param name: 시장환경 이름
        :param pricing_date : 시장 환경 날짜
        """
        self.name = name
        self.pricing_date = pricing_date
        self.constants = {}
        self.lists = {}
        self.curves = {}
        
    def add_constant(self, key, constant):
        """ 상수(모형 파라미터) 추가"""
        self.constants[key] = constant
        
    def get_constant(self, key):
        """ 상수 반환"""
        return self.constants[key]
    
    def add_list(self, key, list_object):
        """ (기초자산 등의) 리스트 추가"""
        self.lists[key] = list_object
        
    def get_list(self, key):
        """ 리스트 반환 """
        return self.lists[key]
    
    def add_curve(self, key, curve):
        """ (이자율 커브 등의) 시장 커브 추가"""
        self.curves[key] = curve
        
    def get_curve(self, key):
        """ 시장 커브 반환"""
        return self.curves[key]
    
    def add_environment(self, env):
        """ 상수, 리스트 ,커브 등의 전체 시장 환경을 추가하거나 갱신"""
        
        for key in env.constants:
            self.constants[key] = env.constants[key]
        
        for key in env.lists:
            self.lists[key] = env.lists[key]
        
        for key in env.curves:
            self.curves[key] = env.curves[key]